# Prediction Algorithms

The main idea is to predict the next day low, upper, open and close to determine a strategy for the next day

In [33]:
import pandas as pd
from datetime import datetime
from datetime import date 
import numpy as np

from time import sleep
from tqdm.notebook import tqdm, trange

import seaborn as sns
import matplotlib as plt
import plotly.graph_objects as go

from tqdm.notebook import tqdm
import warnings
warnings.filterwarnings("ignore")

In [34]:
#Global Variables
coins=['btc','eth','xrp','ltc','bch','tusd','mana','gnt','bat','dai']

In [35]:
#Opening All databases
data=[]
for i,e in enumerate(coins):
    file=e+'-mxn.csv'
    data.append(pd.read_csv(file))
#Conversion of dates from strin to timestamps
for i in range(len(coins)):
    data[i].date=data[i].date.apply(lambda x: datetime.strptime(x,'%Y-%m-%d'))

In [36]:
#Conversion from strings to numeric
def strnum(x):
    try:
        x=x.replace(',','')
        return float(x)
    except:
        return x

for i in range(len(coins)):
    data[i].open=data[i].open.apply(lambda x: strnum(x))
    data[i].close=data[i].close.apply(lambda x: strnum(x))
    data[i].low=data[i].low.apply(lambda x: strnum(x))
    data[i].high=data[i].high.apply(lambda x: strnum(x))
    data[i].volume=data[i].volume.apply(lambda x: strnum(x))

    
#One hot encoding to date


In [37]:
#-----------movingAvg function-----------------
#Calculates the Moving Average

#df - list with the dataframes
#x - index of the coin, e.g. 0- BTC, 1-ETH... of the databases' list who contains the collections of data
#dname - name of the destination column of the moving average calculation (not creared yet)
#oname - name of the origin column of the data to take the information for the moving avarege
#msize - number of registers to calculate the moving average

def movingAvg(df,x,dname,oname,msize):
    df[x][dname] = 0.0
    for i in range(len(df[x])):
        temp=0.0
        if i<msize:
            pass
        else:
            for j in range(msize):
                temp+= data[x][oname][i-msize+j]
            df[x][dname][i]=temp/msize

In [38]:
#-----------------oheDate--------------------
#Makes one hot econfing to date depending day of the week, day of the month, week of year and month of year

def oheDate(df,x):
    df[x]['weekday']=df[x]['date']
    df[x]['weekday']=df[x]['weekday'].apply(lambda z: str(z.weekday()))
    
    #df[x]['calendarweek']=df[x]['date']
    #df[x]['calendarweek']=df[x]['date'].apply(lambda z: str(z.isocalendar()[1]))
    
    #df[x]['dayofmonth']=df[x]['date']
    #df[x]['dayofmonth']=df[x]['date'].apply(lambda z: str(z.day))
    
    #df[x]['month']=df[x]['date']
    #df[x]['month']=df[x]['date'].apply(lambda z: str(z.month))
    
    #Contanating dummies
    ohe=pd.get_dummies(df[x][['weekday']])#,'calendarweek']])#,'dayofmonth','month']])
    df[x]=pd.concat([df[x],ohe],axis=1)
    #Eliminating categorical columns
    df[x].drop(columns=['weekday'], inplace=True)
    

In [39]:
metricas=['open','close','high','low','volume']

for i in tqdm(range(len(coins))):
    for m in metricas:#tqdm(metricas):
        movingAvg(data,i,'PMEF'+ m,m,12)
        movingAvg(data,i,'PMES'+m,m,26)
        data[i]['MACD'+m]=data[i]['PMEF'+m]-data[i]['PMES'+m]
        movingAvg(data,i,'MACD9'+m,'MACD'+m,9)
        data[i]['diffMACD'+m]=data[i]['MACD'+m]-data[i]['MACD9'+m]
    oheDate(data,i)


In [40]:
from sklearn.model_selection import train_test_split as tts
from xgboost import XGBRegressor as XGBR
from sklearn.ensemble import RandomForestRegressor as RFR
from sklearn.ensemble import GradientBoostingRegressor as GBR

In [41]:
#Create Dataframe for predictions
preds= pd.DataFrame(columns=metricas)
preds['coin'] =""
cols=preds.columns.tolist()
cols=cols[-1:]+cols[:-1]
preds=preds[cols]
preds

for x in tqdm(range(len(coins))):
    test=data[x][35:len(data[x])-1] #35 is 26 + 9 for all the number of samples who doesn't has Moving Avg  
    dc=['date','open','close','low','high','volume']
    timevars = [ 'weekday_'+ str(x) for x in range(7)]
    #Adding new row for predictions
    preds=preds.append([0], ignore_index=True).drop(0,axis=1)
    preds=preds[cols]
    preds['coin'][x]=coins[x]
    for m in metricas:
        #Predictions
        X_train, X_test, y_train, y_test=tts(test[['PMEF'+m,'PMES'+m,'MACD'+m,'MACD9'+m]+timevars],test[m], shuffle=True,random_state=100,test_size=.2)
        modelo=XGBR()
        modelo.fit(X_train, y_train)
        preds[m][x]=modelo.predict(data[x][len(data[x])-1:][['PMEF'+m,'PMES'+m,'MACD'+m,'MACD9'+m]+timevars])[0]
        #train_score=modelo.score(X_train, y_train)
        #test_score=modelo.score(X_test, y_test)
        #print ('Score para', coins[x],m,train_score, test_score)

#Getting oppenings to make initial corrections
preds['ropen']=0.0
for i,e in enumerate(coins):
    preds.ropen[i]=data[i]['open'][len(data[i])-1]

preds['corr']=preds.ropen - preds.open
        
#Calculation of investment factors
preds['go']= preds.close - preds.open
preds['go']= preds['go'].apply(lambda x: 1 if x>0 else 0)
preds['ent']=preds.open - (preds.open-preds.low)/2
preds['out']=(preds.high-preds.close)/2+preds.close
preds['rate']=(preds.out/preds.ent-1)*100
preds.go=preds.go*preds.rate
preds.go=preds['go'].apply(lambda x: 1 if x>0 else 0)
preds.rate=preds.rate.apply(lambda x: x.round(1))

preds['enta'] = preds['ent'] + preds['corr']
preds['outa'] = preds['out'] + preds['corr']
preds['rrate']= (preds['outa']/preds['enta'] - 1)*100
preds['stoploss'] = preds['low'] + preds['corr']
preds.rrate=preds.rrate.apply(lambda x: x.round(1))

In [42]:
sortedcols=['coin','enta','outa','stoploss','go','rrate']
preds[sortedcols]

,coin,enta,outa,stoploss,go,rrate
0,btc,208744,210495,206883,0,0.8
1,eth,5170.87,5254.88,5142.15,0,1.6
2,xrp,4.02294,4.1602,3.99589,1,3.4
3,ltc,925.491,925.847,896.633,0,0.0
4,bch,5341.04,5601.33,5518.94,1,4.9
5,tusd,22.663,22.5812,22.3759,0,-0.4
6,mana,0.891999,0.94959,0.893999,1,6.5
7,gnt,1.2075,1.19072,1.23499,0,-1.4
8,bat,5.9307,6.3851,6.00141,1,7.7
9,dai,23.3684,23.3022,23.3568,0,-0.3


In [43]:
preds

,coin,open,close,high,low,volume,ropen,corr,go,ent,out,rate,enta,outa,rrate,stoploss
0,btc,211579,209631,213305,207856,258.477,210606.21,-973.118,0,209717,211468,0.8,208744,210495,0.8,206883
1,eth,5203.37,5173.89,5343.46,5145.95,1214.29,5199.58,-3.79256,0,5174.66,5258.68,1.6,5170.87,5254.88,1.6,5142.15
2,xrp,3.87453,3.93691,4.03254,3.82041,1.35666e+07,4.05,0.175475,1,3.84747,3.98472,3.6,4.02294,4.1602,3.4,3.99589
3,ltc,976.625,917.278,978.965,918.907,737.048,954.35,-22.2746,0,947.766,948.122,0.0,925.491,925.847,0.0,896.633
4,bch,4887.03,5173.9,5476.53,5242.82,212.905,5163.15,276.117,1,5064.93,5325.22,5.1,5341.04,5601.33,4.9,5518.94
5,tusd,23.5061,23.0384,23.2362,22.932,50851,22.95,-0.556126,0,23.2191,23.1373,-0.4,22.663,22.5812,-0.4,22.3759
6,mana,0.909325,0.922268,1.01556,0.913324,2.73047e+06,0.89,-0.0193254,1,0.911325,0.968916,6.3,0.891999,0.94959,6.5,0.893999
7,gnt,1.2292,1.23977,1.24007,1.28419,265158,1.18,-0.0491995,0,1.2567,1.23992,-1.3,1.2075,1.19072,-1.4,1.23499
8,bat,6.35001,6.78329,6.96695,6.49142,148899,5.86,-0.490012,1,6.42071,6.87512,7.1,5.9307,6.3851,7.7,6.00141
9,dai,23.1211,23.0831,23.0037,23.0979,14733.6,23.38,0.258864,0,23.1095,23.0434,-0.3,23.3684,23.3022,-0.3,23.3568


# Graphic

In [ ]:
#Graphic with Candle Sticks
i=8
fig=go.Figure(data=[go.Candlestick(x=data[i].date,
                                   open=data[i].open,
                                   high=data[i].high,
                                   low=data[i].low,
                                   close=data[i].close
                                  )
                   ]
             )
fig.show()